Malicious or Benign Websites EDA 

Chris Heimbuch


In [575]:
# Standard DS imports
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Inferential Analysis Tests
from scipy import stats
from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
from scipy.stats import ttest_ind
from scipy.stats import f_oneway

#For null value imputation
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

In [576]:
# Call in dataset and inspect the head.
df = pd.read_csv("dataset.csv")
df.head()


,URL,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CHARSET,SERVER,CONTENT_LENGTH,WHOIS_COUNTRY,WHOIS_STATEPRO,WHOIS_REGDATE,WHOIS_UPDATED_DATE,...,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type
0,M0_109,16,7,iso-8859-1,nginx,263.0,NaN,NaN,10/10/2015 18:21,NaN,...,0,2,700,9,10,1153,832,9,2.0,1
1,B0_2314,16,6,UTF-8,Apache/2.4.10,15087.0,NaN,NaN,NaN,NaN,...,7,4,1230,17,19,1265,1230,17,0.0,0
2,B0_911,16,6,us-ascii,Microsoft-HTTPAPI/2.0,324.0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0
3,B0_113,17,6,ISO-8859-1,nginx,162.0,US,AK,7/10/1997 4:00,12/09/2013 0:45,...,22,3,3812,39,37,18784,4380,39,8.0,0
4,B0_403,17,6,UTF-8,NaN,124140.0,US,TX,12/05/1996 0:00,11/04/2017 0:00,...,2,5,4278,61,62,129889,4586,61,4.0,0


In [578]:
#Inspect dimensionality of dataset.
print(df.shape)

(1781, 21)


In [579]:
#Inspect the column names and see what we are working with.
df.columns

Index(['URL', 'URL_LENGTH', 'NUMBER_SPECIAL_CHARACTERS', 'CHARSET', 'SERVER',
       'CONTENT_LENGTH', 'WHOIS_COUNTRY', 'WHOIS_STATEPRO', 'WHOIS_REGDATE',
       'WHOIS_UPDATED_DATE', 'TCP_CONVERSATION_EXCHANGE',
       'DIST_REMOTE_TCP_PORT', 'REMOTE_IPS', 'APP_BYTES', 'SOURCE_APP_PACKETS',
       'REMOTE_APP_PACKETS', 'SOURCE_APP_BYTES', 'REMOTE_APP_BYTES',
       'APP_PACKETS', 'DNS_QUERY_TIMES', 'Type'],
      dtype='object')

In [580]:
# Call info to inspect data types and get a preliminary investigtion of any null values. It appears we have some.
df.info()

print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1781 entries, 0 to 1780
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   URL                        1781 non-null   object 
 1   URL_LENGTH                 1781 non-null   int64  
 2   NUMBER_SPECIAL_CHARACTERS  1781 non-null   int64  
 3   CHARSET                    1774 non-null   object 
 4   SERVER                     1605 non-null   object 
 5   CONTENT_LENGTH             969 non-null    float64
 6   WHOIS_COUNTRY              1475 non-null   object 
 7   WHOIS_STATEPRO             1419 non-null   object 
 8   WHOIS_REGDATE              1654 non-null   object 
 9   WHOIS_UPDATED_DATE         1642 non-null   object 
 10  TCP_CONVERSATION_EXCHANGE  1781 non-null   int64  
 11  DIST_REMOTE_TCP_PORT       1781 non-null   int64  
 12  REMOTE_IPS                 1781 non-null   int64  
 13  APP_BYTES                  1781 non-null   int64

In [581]:
#Seeing how many null values I have and in what columns.
df.isna().sum()

URL                            0
URL_LENGTH                     0
NUMBER_SPECIAL_CHARACTERS      0
CHARSET                        7
SERVER                       176
CONTENT_LENGTH               812
WHOIS_COUNTRY                306
WHOIS_STATEPRO               362
WHOIS_REGDATE                127
WHOIS_UPDATED_DATE           139
TCP_CONVERSATION_EXCHANGE      0
DIST_REMOTE_TCP_PORT           0
REMOTE_IPS                     0
APP_BYTES                      0
SOURCE_APP_PACKETS             0
REMOTE_APP_PACKETS             0
SOURCE_APP_BYTES               0
REMOTE_APP_BYTES               0
APP_PACKETS                    0
DNS_QUERY_TIMES                1
Type                           0
dtype: int64

In [582]:
# Simple Statistical summary of data set with numerical values.
df.describe()

,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CONTENT_LENGTH,TCP_CONVERSATION_EXCHANGE,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type
count,1781.000000,1781.000000,969.000000,1781.000000,1781.000000,1781.000000,1.781000e+03,1781.000000,1781.000000,1.781000e+03,1.781000e+03,1781.000000,1780.000000,1781.000000
mean,56.961258,11.111735,11726.927761,16.261089,5.472768,3.060640,2.982339e+03,18.540146,18.746210,1.589255e+04,3.155599e+03,18.540146,2.263483,0.121280
std,27.555586,4.549896,36391.809051,40.500975,21.807327,3.386975,5.605057e+04,41.627173,46.397969,6.986193e+04,5.605378e+04,41.627173,2.930853,0.326544
min,16.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,39.000000,8.000000,324.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
50%,49.000000,10.000000,1853.000000,7.000000,0.000000,2.000000,6.720000e+02,8.000000,9.000000,5.790000e+02,7.350000e+02,8.000000,0.000000,0.000000
75%,68.000000,13.000000,11323.000000,22.000000,5.000000,5.000000,2.328000e+03,26.000000,25.000000,9.806000e+03,2.701000e+03,26.000000,4.000000,0.000000
max,249.000000,43.000000,649263.000000,1194.000000,708.000000,17.000000,2.362906e+06,1198.000000,1284.000000,2.060012e+06,2.362906e+06,1198.000000,20.000000,1.000000


In [583]:
df

,URL,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CHARSET,SERVER,CONTENT_LENGTH,WHOIS_COUNTRY,WHOIS_STATEPRO,WHOIS_REGDATE,WHOIS_UPDATED_DATE,...,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type
0,M0_109,16,7,iso-8859-1,nginx,263.0,NaN,NaN,10/10/2015 18:21,NaN,...,0,2,700,9,10,1153,832,9,2.0,1
1,B0_2314,16,6,UTF-8,Apache/2.4.10,15087.0,NaN,NaN,NaN,NaN,...,7,4,1230,17,19,1265,1230,17,0.0,0
2,B0_911,16,6,us-ascii,Microsoft-HTTPAPI/2.0,324.0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0
3,B0_113,17,6,ISO-8859-1,nginx,162.0,US,AK,7/10/1997 4:00,12/09/2013 0:45,...,22,3,3812,39,37,18784,4380,39,8.0,0
4,B0_403,17,6,UTF-8,NaN,124140.0,US,TX,12/05/1996 0:00,11/04/2017 0:00,...,2,5,4278,61,62,129889,4586,61,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1776,M4_48,194,16,UTF-8,Apache,NaN,ES,Barcelona,17/09/2008 0:00,2/09/2016 0:00,...,0,0,0,0,3,186,0,0,0.0,1
1777,M4_41,198,17,UTF-8,Apache,NaN,ES,Barcelona,17/09/2008 0:00,2/09/2016 0:00,...,0,0,0,0,2,124,0,0,0.0,1
1778,B0_162,201,34,utf-8,Apache/2.2.16 (Debian),8904.0,US,FL,15/02/1999 0:00,15/07/2015 0:00,...,2,6,6631,87,89,132181,6945,87,4.0,0
1779,B0_1152,234,34,ISO-8859-1,cloudflare-nginx,NaN,US,CA,1/04/1998 0:00,9/12/2016 0:00,...,0,0,0,0,0,0,0,0,0.0,0


Just a quick preliminary review of my data, it is messy. Columns are uppercase with the last column being title case, there are null values across many rows with some reaching over 800 null values, string data has dashes and certain strings that are the same name such as UTF-8 in the "CHARSET" column are upper case and lower case. Therefore, I will add some consistency across my data to prepare for the exploratory data analysis.

In [584]:
#Going to create a copy of the df to work with.
df_copy = df.copy()

In [585]:
#Make all columns lowercase
df_copy.columns = df_copy.columns.str.lower()

In [586]:
#Clean up the 'charset' column: replace the dashes with spaces and make it Title case
df_copy['charset'] = df_copy['charset'].str.replace('-', ' ')
df_copy['charset'] = df_copy['charset'].str.title()
df_copy['charset'].unique()

array(['Iso 8859 1', 'Utf 8', 'Us Ascii', nan, 'Windows 1251', 'Iso 8859',
       'Windows 1252'], dtype=object)

In [587]:
#Going to perform null value imputation with KNN imputation method.

#Encode only the known values.
le = LabelEncoder()
df_copy['charset_encoded'] = df_copy['charset']
df_copy.loc[df_copy['charset'].notnull(), 'charset_encoded'] = le.fit_transform(df_copy['charset'].dropna())

#Apply KNN imputation on the encoded column.
imputer = KNNImputer(n_neighbors=3)
df_copy['charset_encoded'] = imputer.fit_transform(df_copy[['charset_encoded']])

#Inverse transform the encoded data back to the original categories.
df_copy['charset_imputed'] = df_copy['charset_encoded'].round().astype(int)
df_copy['charset_imputed'] = le.inverse_transform(df_copy['charset_imputed'])

#Compare the original and imputed columns to verify our imputation has succeeded.
imputed_values = df_copy[df_copy['charset'].isnull()]

print(imputed_values[['charset', 'charset_imputed']])

     charset charset_imputed
35       NaN        Us Ascii
81       NaN        Us Ascii
125      NaN        Us Ascii
159      NaN        Us Ascii
952      NaN        Us Ascii
977      NaN        Us Ascii
1069     NaN        Us Ascii


In [588]:
df_copy = df_copy.drop(columns=["charset_encoded", "charset"])

In [589]:
df_copy.isna().sum()

url                            0
url_length                     0
number_special_characters      0
server                       176
content_length               812
whois_country                306
whois_statepro               362
whois_regdate                127
whois_updated_date           139
tcp_conversation_exchange      0
dist_remote_tcp_port           0
remote_ips                     0
app_bytes                      0
source_app_packets             0
remote_app_packets             0
source_app_bytes               0
remote_app_bytes               0
app_packets                    0
dns_query_times                1
type                           0
charset_imputed                0
dtype: int64

In [590]:
#Looking into "server" column now. There is alot of random noise here.
df_copy['server'].unique()

array(['nginx', 'Apache/2.4.10', 'Microsoft-HTTPAPI/2.0', nan, 'Apache/2',
       'nginx/1.10.1', 'Apache', 'Apache/2.2.15 (Red Hat)',
       'Apache/2.4.23 (Unix) OpenSSL/1.0.1e-fips mod_bwlimited/1.4',
       'openresty/1.11.2.1', 'Apache/2.2.22', 'Apache/2.4.7 (Ubuntu)',
       'nginx/1.12.0',
       'Apache/2.4.12 (Unix) OpenSSL/1.0.1e-fips mod_bwlimited/1.4',
       'Oracle-iPlanet-Web-Server/7.0', 'cloudflare-nginx', 'nginx/1.6.2',
       'openresty', 'Heptu web server', 'Pepyaka/1.11.3', 'nginx/1.8.0',
       'nginx/1.10.1 + Phusion Passenger 5.0.30',
       'Apache/2.2.29 (Amazon)', 'Microsoft-IIS/7.5', 'LiteSpeed',
       'Apache/2.4.25 (cPanel) OpenSSL/1.0.1e-fips mod_bwlimited/1.4',
       'tsa_c', 'Apache/2.2.0 (Fedora)', 'Apache/2.2.22 (Debian)',
       'Apache/2.2.15 (CentOS)', 'Apache/2.4.25',
       'Apache/2.4.25 (Amazon) PHP/7.0.14', 'GSE',
       'Apache/2.4.23 (Unix) OpenSSL/0.9.8e-fips-rhel5 mod_bwlimited/1.4',
       'Apache/2.4.25 (Amazon) OpenSSL/1.0.1k-fips',
 

In [591]:
df_copy['server'].fillna("Unknown", inplace=True)

In [592]:
#Clean up the 'charset' column: replace the dashes with spaces and make it Title case
# df_copy['server'] = df_copy['server'].str.replace('-', ' ')
df_copy['server'] = df_copy['server'].str.lower()

In [593]:
df_copy['standardized_server'] = df_copy['server'].apply(standardize_server)

print(df_copy[['server', 'standardized_server']])

                      server standardized_server
0                      nginx               nginx
1              apache/2.4.10              apache
2      microsoft-httpapi/2.0       microsoft-IIS
3                      nginx               nginx
4                    unknown             unknown
...                      ...                 ...
1776                  apache              apache
1777                  apache              apache
1778  apache/2.2.16 (debian)              apache
1779        cloudflare-nginx               nginx
1780       microsoft-iis/8.5       microsoft-IIS

[1781 rows x 2 columns]


In [594]:
df_copy['server'].unique()

array(['nginx', 'apache/2.4.10', 'microsoft-httpapi/2.0', 'unknown',
       'apache/2', 'nginx/1.10.1', 'apache', 'apache/2.2.15 (red hat)',
       'apache/2.4.23 (unix) openssl/1.0.1e-fips mod_bwlimited/1.4',
       'openresty/1.11.2.1', 'apache/2.2.22', 'apache/2.4.7 (ubuntu)',
       'nginx/1.12.0',
       'apache/2.4.12 (unix) openssl/1.0.1e-fips mod_bwlimited/1.4',
       'oracle-iplanet-web-server/7.0', 'cloudflare-nginx', 'nginx/1.6.2',
       'openresty', 'heptu web server', 'pepyaka/1.11.3', 'nginx/1.8.0',
       'nginx/1.10.1 + phusion passenger 5.0.30',
       'apache/2.2.29 (amazon)', 'microsoft-iis/7.5', 'litespeed',
       'apache/2.4.25 (cpanel) openssl/1.0.1e-fips mod_bwlimited/1.4',
       'tsa_c', 'apache/2.2.0 (fedora)', 'apache/2.2.22 (debian)',
       'apache/2.2.15 (centos)', 'apache/2.4.25',
       'apache/2.4.25 (amazon) php/7.0.14', 'gse',
       'apache/2.4.23 (unix) openssl/0.9.8e-fips-rhel5 mod_bwlimited/1.4',
       'apache/2.4.25 (amazon) openssl/1.0.1k-fi

In [595]:
#Grouping values with the least count into one bin "Other" to reduce number of unique values
series = pd.value_counts(df_copy.server)
mask = (series/series.sum() * 100).lt(1)
df_copy['server'] = np.where(df_copy['server'].isin(series[mask].index),'other',df_copy['server'])

In [596]:
df_copy['server'].unique()

array(['nginx', 'other', 'microsoft-httpapi/2.0', 'unknown', 'apache',
       'nginx/1.12.0', 'cloudflare-nginx', 'microsoft-iis/7.5',
       'apache/2.2.15 (centos)', 'gse', 'ats', 'server',
       'youtubefrontend', 'apache-coyote/1.1'], dtype=object)

In [597]:
 # Further bucketing and cleaning up the server column.

def standardize_server(server_string):
    if 'apache' in server_string:
        return 'apache'
    if 'nginx' in server_string:
        return 'nginx'
    if 'microsoft' in server_string:
        return 'microsoft-IIS'

    return server_string


df_copy['standardized_server'] = df_copy['server'].apply(standardize_server)
df_copy['standardized_server'] = df_copy['standardized_server'].str.replace('-', ' ')

print(df_copy[['server', 'standardized_server']])

                     server standardized_server
0                     nginx               nginx
1                     other               other
2     microsoft-httpapi/2.0       microsoft IIS
3                     nginx               nginx
4                   unknown             unknown
...                     ...                 ...
1776                 apache              apache
1777                 apache              apache
1778                  other               other
1779       cloudflare-nginx               nginx
1780                  other               other

[1781 rows x 2 columns]


In [598]:
#Final inspection of data.

df_copy['standardized_server'].unique()

array(['nginx', 'other', 'microsoft IIS', 'unknown', 'apache', 'gse',
       'ats', 'server', 'youtubefrontend'], dtype=object)

In [599]:
#Inspecting other columns to clean.
df_copy.isna().sum()

url                            0
url_length                     0
number_special_characters      0
server                         0
content_length               812
whois_country                306
whois_statepro               362
whois_regdate                127
whois_updated_date           139
tcp_conversation_exchange      0
dist_remote_tcp_port           0
remote_ips                     0
app_bytes                      0
source_app_packets             0
remote_app_packets             0
source_app_bytes               0
remote_app_bytes               0
app_packets                    0
dns_query_times                1
type                           0
charset_imputed                0
standardized_server            0
dtype: int64

In [600]:
#Content length has the most null values and has a very large spread. Filling with the mean could work but then it would heavily skew the column since a given website may not exactly have 11726 like the mean.
#The median may also not work well since its 1853, and imputing with KNN may not work well. Instead, I will interpolate the data so it is more linear and equally spaced, to make it more realistic as not every website is equal.
df_copy[['content_length']].describe()

,content_length
count,969.000000
mean,11726.927761
std,36391.809051
min,0.000000
25%,324.000000
50%,1853.000000
75%,11323.000000
max,649263.000000


In [601]:
df_copy['content_length'] = df_copy['content_length'].interpolate()

In [602]:
df_copy[['content_length']].describe()

,content_length
count,1781.000000
mean,13497.243964
std,38415.552697
min,0.000000
25%,603.000000
50%,4714.750000
75%,12578.500000
max,649263.000000


In [603]:
#Inspecting other columns to clean and seeing if we filled in content_length successfully.
df_copy.isna().sum()

url                            0
url_length                     0
number_special_characters      0
server                         0
content_length                 0
whois_country                306
whois_statepro               362
whois_regdate                127
whois_updated_date           139
tcp_conversation_exchange      0
dist_remote_tcp_port           0
remote_ips                     0
app_bytes                      0
source_app_packets             0
remote_app_packets             0
source_app_bytes               0
remote_app_bytes               0
app_packets                    0
dns_query_times                1
type                           0
charset_imputed                0
standardized_server            0
dtype: int64

In [604]:
#Inspecting the whois_country. There is some cleaning that should be done - multiple for the UK, se lowercase, nulls.
df_copy['whois_country'].unique()

array([nan, 'US', 'SC', 'GB', 'UK', 'RU', 'AU', 'CA', 'PA', 'se', 'IN',
       'LU', 'TH', "[u'GB'; u'UK']", 'FR', 'NL', 'UG', 'JP', 'CN', 'SE',
       'SI', 'IL', 'ru', 'KY', 'AT', 'CZ', 'PH', 'BE', 'NO', 'TR', 'LV',
       'DE', 'ES', 'BR', 'us', 'KR', 'HK', 'UA', 'CH', 'United Kingdom',
       'BS', 'PK', 'IT', 'Cyprus', 'BY', 'AE', 'IE', 'UY', 'KG'],
      dtype=object)

In [605]:
df_copy['whois_country'].value_counts()

whois_country
US                1103
CA                  84
ES                  63
AU                  35
PA                  21
GB                  19
JP                  11
UK                  10
CN                  10
IN                  10
FR                   9
CZ                   9
NL                   6
CH                   6
[u'GB'; u'UK']       5
KR                   5
PH                   4
BS                   4
ru                   4
AT                   4
HK                   3
us                   3
TR                   3
BE                   3
DE                   3
SC                   3
KY                   3
SE                   3
BR                   2
UY                   2
Cyprus               2
SI                   2
UA                   2
RU                   2
IL                   2
NO                   2
KG                   2
TH                   1
se                   1
LV                   1
LU                   1
United Kingdom       1
UG                  

In [606]:
#Function to replace the strange values in the column.
def replace(x):
    if x == "[u'GB'; u'UK']"or x=="United Kingdom" or x=="UK":
        return "GB"
    elif x == "Cyprus":
        return "CY"
    elif x == "us":
        return "US"
    elif x == "ru":
        return "RU"
    elif x == "se":
        return "SE"
    else:
        return x
    
df_copy["whois_country"] = list(map(lambda x: replace(x), df_copy["whois_country"]))

In [607]:
#Sanity check
df_copy['whois_country'].unique()

array([nan, 'US', 'SC', 'GB', 'RU', 'AU', 'CA', 'PA', 'SE', 'IN', 'LU',
       'TH', 'FR', 'NL', 'UG', 'JP', 'CN', 'SI', 'IL', 'KY', 'AT', 'CZ',
       'PH', 'BE', 'NO', 'TR', 'LV', 'DE', 'ES', 'BR', 'KR', 'HK', 'UA',
       'CH', 'BS', 'PK', 'IT', 'CY', 'BY', 'AE', 'IE', 'UY', 'KG'],
      dtype=object)

In [608]:
#Filling the NA as 'other' category.
df_copy['whois_country'].fillna("Other", inplace=True)

In [609]:
#Sanity check
df_copy['whois_country'].unique()

array(['Other', 'US', 'SC', 'GB', 'RU', 'AU', 'CA', 'PA', 'SE', 'IN',
       'LU', 'TH', 'FR', 'NL', 'UG', 'JP', 'CN', 'SI', 'IL', 'KY', 'AT',
       'CZ', 'PH', 'BE', 'NO', 'TR', 'LV', 'DE', 'ES', 'BR', 'KR', 'HK',
       'UA', 'CH', 'BS', 'PK', 'IT', 'CY', 'BY', 'AE', 'IE', 'UY', 'KG'],
      dtype=object)

In [610]:
#Inspecting other columns to clean and seeing if we filled in whois_country successfully.
df_copy.isna().sum()

url                            0
url_length                     0
number_special_characters      0
server                         0
content_length                 0
whois_country                  0
whois_statepro               362
whois_regdate                127
whois_updated_date           139
tcp_conversation_exchange      0
dist_remote_tcp_port           0
remote_ips                     0
app_bytes                      0
source_app_packets             0
remote_app_packets             0
source_app_bytes               0
remote_app_bytes               0
app_packets                    0
dns_query_times                1
type                           0
charset_imputed                0
standardized_server            0
dtype: int64

In [611]:
#statepro
df_copy['whois_statepro'].unique()

array([nan, 'AK', 'TX', 'Mahe', 'CO', 'FL', 'Kansas',
       'Novosibirskaya obl.', 'CA', 'Tennessee', 'Vi', 'OR', 'Texas',
       'ALBERTA', 'PANAMA', 'Arizona', 'WI', 'Oregon', 'Andhra Pradesh',
       'AB', 'Tamil Nadu', 'VA', 'NY', 'quebec', 'MA', 'ON', 'New Mexico',
       'British Columbia', 'Massachusetts', 'California', 'bangkok',
       'WEST MIDLANDS', 'TEXAS', 'WC1N', 'Kentucky', 'MD', 'NEW YORK',
       'Washington', 'Colorado', 'PA', 'LA', 'WA', 'Queensland', 'MOSCOW',
       'UK', 'P', 'NH', 'Pennsylvania', 'UTTAR PRADESH', 'NC', 'kireka',
       'IL', 'Missouri', 'Osaka', 'QC', 'Michigan', 'Maryland', 'Ontario',
       'South Carolina', 'Zhejiang', 'New York', 'QLD', 'NJ', 'GA', 'MO',
       'HR', 'ab', 'Greater London', 'Illinois', '--', 'Fukuoka', 'BC',
       'AL', 'Krasnoyarsk', 'MAINE', 'Virginia', 'MH', 'GRAND CAYMAN',
       'Austria', 'DE', 'shandong', 'AZ', 'PRAHA', 'beijingshi',
       'liaoningsheng', 'North Carolina', 'OH', 'Manila', 'Utah', 'MI',
       'NSW

In [612]:
#Cleaning up data and nulls.
def replace_state(x):
    if x == "California"or x=="CALIFORNIA":
        return "CA"
    elif x == "Arizona":
        return "AZ"
    elif x == "New York" or x=="NEW YORK":
        return "NY"
    elif x == "Ohio":
        return "OH"
    elif x == "Utah":
        return "UT"
    elif x == "None":
        return "NA"
    elif x == "Texas":
        return "TX"
    elif x == "Washington":
        return "WA"
    elif x == "va":
        return "VA"
    elif x == "Illinois" or x=="il":
        return "IL"
    elif x == "District of Columbia" or x=="DC" or x=="Maryland":
        return "MD"
    elif x == "New Jersey":
        return "NJ"
    elif x == "Maine" or x=="MAINE":
        return "ME"
    elif x == "Quebec" or x=="QUEBEC" or x=="qc" or x=="quebec":
        return "QC"
    elif x == "Missouri":
        return "MO"
    elif x == "Nevada":
        return "NV"
    elif x == "WC1N" or x=="Greater London" or x=="UK" or x=="WEST MIDLANDS" or x=="worcs" or x=="Peterborough" or x=="London" or x=="HANTS" or x=="MIDDLESEX":
        return "England"
    elif x == "Pennsylvania":
        return "PA"
    elif x == "Florida" or x=="FLORIDA":
        return "FL"
    elif x == "PANAMA":
        return "Panama"
    else:
        return x
    
df_copy["whois_statepro"] = list(map(lambda x: replace_state(x), df_copy["whois_statepro"]))

In [613]:
df_copy["whois_statepro"].value_counts()[:21]

whois_statepro
CA           430
NY            87
WA            75
FL            67
AZ            64
Barcelona     62
ON            45
UT            42
NV            33
IL            28
PA            28
CO            24
England       22
MO            22
MA            22
Panama        21
OH            21
TX            19
VA            18
NJ            16
QC            15
Name: count, dtype: int64

In [614]:
#Grouping values with the least count into one bin "Other" to reduce number of unique values
counts = df_copy['whois_statepro'].value_counts()
df_copy['whois_statepro'] = np.where(df_copy['whois_statepro'].isin(counts[counts < 15].index),'Other',df_copy['whois_statepro'])

In [615]:
df_copy['whois_statepro'].fillna("Other", inplace=True)

In [616]:
df_copy['whois_statepro'].unique()

array(['Other', 'TX', 'CO', 'FL', 'CA', 'Panama', 'AZ', 'VA', 'NY', 'QC',
       'MA', 'ON', 'England', 'WA', 'PA', 'IL', 'MO', 'NJ', 'OH', 'UT',
       'NV', 'Barcelona'], dtype=object)

In [617]:
#Inspecting other columns to clean and seeing if we filled in whois_country successfully.
df_copy.isna().sum()

url                            0
url_length                     0
number_special_characters      0
server                         0
content_length                 0
whois_country                  0
whois_statepro                 0
whois_regdate                127
whois_updated_date           139
tcp_conversation_exchange      0
dist_remote_tcp_port           0
remote_ips                     0
app_bytes                      0
source_app_packets             0
remote_app_packets             0
source_app_bytes               0
remote_app_bytes               0
app_packets                    0
dns_query_times                1
type                           0
charset_imputed                0
standardized_server            0
dtype: int64

In [618]:
#Inspect the regdate column
df_copy['whois_regdate'].value_counts()

whois_regdate
17/09/2008 0:00    62
13/01/2001 0:12    59
31/07/2000 0:00    47
15/02/2005 0:00    41
29/03/1997 0:00    33
                   ..
23/11/1994 0:00     1
30/08/2015 0:00     1
30/04/2009 0:00     1
27/11/2006 0:00     1
14/11/2008 0:00     1
Name: count, Length: 890, dtype: int64

In [619]:
df_copy['whois_regdate'].unique()

array(['10/10/2015 18:21', nan, '7/10/1997 4:00', '12/05/1996 0:00',
       '3/08/2016 14:30', '29/07/2002 0:00', '18/03/1997 0:00',
       '8/11/2014 7:41', '14/09/2007 0:00', '22/11/2016 0:00',
       '11/10/2002 0:00', '14/11/2002 0:00', '16/07/2000 0:00',
       '25/05/2013 0:00', '9/08/1999 0:00', '15/09/2013 0:00',
       '3/07/1999 0:00', '2/11/2003 0:00', '12/08/2008 22:10',
       '21/05/2009 0:00', '1/08/2002 0:00', '13/01/2005 0:00',
       '18/05/2005 19:41', '4/01/2001 0:00', '28/02/2008 10:58',
       '8/12/2006 0:00', '16/06/2000 0:00', '13/10/2000 0:00',
       '31/12/1999 0:00', '30/07/1996 0:00', '9/05/2008 0:00',
       '23/04/1999 0:00', '4/02/1997 0:00', '13/02/2003 0:00',
       '17/05/2008 0:00', '30/05/2002 0:00', '20/10/2005 0:00',
       '7/01/2006 0:00', '5/03/1996 5:00', '23/03/1995 0:00',
       '10/01/1998 0:00', '27/04/2016 0:00', '7/04/2011 0:00',
       '26/02/2009 0:00', '3/07/2002 0:00', '21/02/1995 0:00',
       '4/07/2007 0:00', '2/07/1998 0:00', '1

In [620]:
#Going to look at the dates now and clean up the format on those.

def date_cleaner(datetime_str):
    if datetime_str in [np.nan, "b", "0", "None"]:  # these are the missing values and a random b thrown in the mix
        return np.nan
    
    if "T" in datetime_str:
        split_datetime = datetime_str.split("T")
    else:
        split_datetime = datetime_str.split()
    
    date = split_datetime[0]
    date_with_slash = date.replace("-", "/")
    
    if date_with_slash == "2002/03/20":  # this is the only instance that doesn't follow the convention
        date_with_slash = "20/03/2002"
    return date_with_slash

In [621]:
#Going to apply the cleaner format to both regdate and updated date columns
df_copy.whois_regdate = df_copy.whois_regdate.apply(date_cleaner)
df_copy["whois_regdate"] = pd.to_datetime(df_copy.whois_regdate, format="%d/%m/%Y", errors="coerce")

#Update the updated_date column
df_copy.whois_updated_date = df_copy.whois_updated_date.apply(date_cleaner)
df_copy["whois_updated_date"] = pd.to_datetime(df_copy.whois_updated_date, format="%d/%m/%Y", errors="coerce")

#Filling null values with the median. The reason being is it is impossible to know what these dates could be. The median offers the most common date which could help with our analysis.
df_copy["whois_regdate"].fillna(df_copy["whois_regdate"].median(), inplace=True)
df_copy["whois_updated_date"].fillna(df_copy["whois_updated_date"].median(), inplace=True)

In [622]:
#Deal with 1 last null value in dns_query_times. 
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1781 entries, 0 to 1780
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   url                        1781 non-null   object        
 1   url_length                 1781 non-null   int64         
 2   number_special_characters  1781 non-null   int64         
 3   server                     1781 non-null   object        
 4   content_length             1781 non-null   float64       
 5   whois_country              1781 non-null   object        
 6   whois_statepro             1781 non-null   object        
 7   whois_regdate              1781 non-null   datetime64[ns]
 8   whois_updated_date         1781 non-null   datetime64[ns]
 9   tcp_conversation_exchange  1781 non-null   int64         
 10  dist_remote_tcp_port       1781 non-null   int64         
 11  remote_ips                 1781 non-null   int64         
 12  app_by

In [623]:
#I will interpolate the dns query column. It's one value, so filling in the null value with a linear approach should work.
df_copy['dns_query_times'] = df_copy['dns_query_times'].interpolate()

In [624]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1781 entries, 0 to 1780
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   url                        1781 non-null   object        
 1   url_length                 1781 non-null   int64         
 2   number_special_characters  1781 non-null   int64         
 3   server                     1781 non-null   object        
 4   content_length             1781 non-null   float64       
 5   whois_country              1781 non-null   object        
 6   whois_statepro             1781 non-null   object        
 7   whois_regdate              1781 non-null   datetime64[ns]
 8   whois_updated_date         1781 non-null   datetime64[ns]
 9   tcp_conversation_exchange  1781 non-null   int64         
 10  dist_remote_tcp_port       1781 non-null   int64         
 11  remote_ips                 1781 non-null   int64         
 12  app_by

In [625]:
#drop the server column since we have it standardized and ready to go.
df_copy = df_copy.drop(columns=["server"])

In [626]:
#data is clean and ready to go!
df_copy.head()

,url,url_length,number_special_characters,content_length,whois_country,whois_statepro,whois_regdate,whois_updated_date,tcp_conversation_exchange,dist_remote_tcp_port,...,app_bytes,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,app_packets,dns_query_times,type,charset_imputed,standardized_server
0,M0_109,16,7,263.0,Other,Other,2015-10-10,2016-08-23,7,0,...,700,9,10,1153,832,9,2.0,1,Iso 8859 1,nginx
1,B0_2314,16,6,15087.0,Other,Other,2001-01-16,2016-08-23,17,7,...,1230,17,19,1265,1230,17,0.0,0,Utf 8,other
2,B0_911,16,6,324.0,Other,Other,2001-01-16,2016-08-23,0,0,...,0,0,0,0,0,0,0.0,0,Us Ascii,microsoft IIS
3,B0_113,17,6,162.0,US,Other,1997-10-07,2013-09-12,31,22,...,3812,39,37,18784,4380,39,8.0,0,Iso 8859 1,nginx
4,B0_403,17,6,124140.0,US,TX,1996-05-12,2017-04-11,57,2,...,4278,61,62,129889,4586,61,4.0,0,Utf 8,unknown
